## Assignment 1 Part II - machine reading knowledge graph instance closure

In this part of the Assignment I you will learn about applying logical closure over a knowledge graph creating using predications extracted using machine reading.

This notebook is broken into 3 parts. 

First, you will learn about the rule programming paradigm using CLIPS through a python library called ClipsPy. You will also learn some recmmended practices for writing production rules. 

Second, you will create a domain of discourse that fits your interests using PubMed and eutils. You will query predications for the domain of discourse from SemMedDB.

Finally, you will run transitive and symmetric closer over the. You will also compare results with the predication set prior to closure.


### 1) The rule programming paradigm and recommended practices for writing production rules.

Complete the cells and answer the questions in [the ClipsPy Hands On Tutorial](clipspy-handson-tutorial.ipynb)

### 2) Create a domain of discourse that fits your interest and query for predications.

#### Assign to the list variable between 100 and 300 PMIDs from a biomedical domain that you have a particular interest in. These could be from a collection you have access to or from a new search. For example, you might pick a disease of interest and search for systematic reviews, meta-analyses, or other study types (e.g., animal or in vitro studies). Do not do more than 300 because it will overtax the server later on when it queries SemMedDB.  Be sure to list PMIDs as integer types e.g., 1,2,3. 

In [1]:
seed_pmid_L = [27569062,24283439,22098123,21062266,20518782,17184278,15096145,10333626,7917994,1619641,1842204,22946697,3789005,3712393,4697981,15490149,15490149,11078563,22837569,19047498,16825433,12522802,12494447,12239713,8725790,9147900,8867662,7917994,1358488,1867292,2063896,1915501,2596507,2596506,2596505,3265306,3425595,26293599,23379592,25049390,22946697,21797849,21178588,20735442,20030748,20882035,19206169,19120036,19156172,19376813,18042262,18456719,17661820,17551924,18039946,16439621,16804887,16474404,16372351,15488754,12752577,12529711,12634870,12494447,12529707,12522804,12116271,12384786,11102944,10925386,11078563,9781012,9272711,8725790,1619641,1785648,3265306,3789005,4697981,18039235,17611774,11078563,2149308,25742478,24705357,21500339,18854871,15488754,10528867,21204800,19376813,18039946,26446362,18039946,24101678,27149079,24458522,26150740,17703371,22946697,11078563,18039235,20735442,21495172,21495178,21062266,17056636,26242988,21495174,16906537,19063751,17974516,20186801,21204800,16520774]

In [2]:
import entrezpy.elink.elinker

#### The next two cells obtain PMIDs for articles cited in the PMIDs you chose and also articles that cite the PMIDs you chose. Both cells uses the eutils API with my email and API key. There is only a very tiny difference in the eutils call.

#### Note: the call returns an error if it cannot retrieve citations for a given PMID. You can ignore the errors so long is it is retrieving citations for most PMIDs.

In [3]:
cited_in_pmids = {}
for pmid in seed_pmid_L:      
    print("INFO: pmid {}".format(pmid))
    e = entrezpy.elink.elinker.Elinker('bioinf2071-course-notebook',
                                   'rdb20@pitt.edu',
                                   '2f8f82da766ed382a93ccb66e19b07a4ef09')
    try:
        analyzer = e.inquire({'db':'pubmed',
                              'dbfrom':'pubmed',
                              'linkname' : 'pubmed_pubmed_citedin',
                              'id' : [pmid]}
                            )
    except TypeError:
        print("ERROR: problem retrieving citations for {}".format(pmid))
        continue
        
    r = analyzer.get_result().dump()
    linkunits = [x['linkunits'] for x in r['linksets']]

    for l in linkunits:
        for u in l:
            cited_in_pmids[u['uid']] = None

# print(cited_in_pmids.keys())


INFO: pmid 27569062
INFO: pmid 24283439
INFO: pmid 22098123
INFO: pmid 21062266
INFO: pmid 20518782
INFO: pmid 17184278
ERROR: problem retrieving citations for 17184278
INFO: pmid 15096145
INFO: pmid 10333626
ERROR: problem retrieving citations for 10333626
INFO: pmid 7917994
INFO: pmid 1619641
INFO: pmid 1842204
ERROR: problem retrieving citations for 1842204
INFO: pmid 22946697
INFO: pmid 3789005
INFO: pmid 3712393
INFO: pmid 4697981
INFO: pmid 15490149
INFO: pmid 15490149
INFO: pmid 11078563
INFO: pmid 22837569
INFO: pmid 19047498
INFO: pmid 16825433
INFO: pmid 12522802
INFO: pmid 12494447
ERROR: problem retrieving citations for 12494447
INFO: pmid 12239713
INFO: pmid 8725790
INFO: pmid 9147900
ERROR: problem retrieving citations for 9147900
INFO: pmid 8867662
INFO: pmid 7917994
INFO: pmid 1358488
ERROR: problem retrieving citations for 1358488
INFO: pmid 1867292
INFO: pmid 2063896
INFO: pmid 1915501
INFO: pmid 2596507
INFO: pmid 2596506
INFO: pmid 2596505
INFO: pmid 3265306
INFO: p

In [4]:
citing_pmids = {}
for pmid in seed_pmid_L:      
    print("INFO: pmid {}".format(pmid))
    e = entrezpy.elink.elinker.Elinker('bioinf2071-course-notebook',
                                   'rdb20@pitt.edu',
                                   '2f8f82da766ed382a93ccb66e19b07a4ef09')
    try:
        analyzer = e.inquire({'db':'pubmed',
                              'dbfrom':'pubmed',
                              'linkname' : 'pubmed_pubmed_refs',
                              'id' : [pmid]}
                            )
    except TypeError:
        print("ERROR: problem retrieving citations for {}".format(pmid))
        continue 
        
    r = analyzer.get_result().dump()
    linkunits = [x['linkunits'] for x in r['linksets']]

    for l in linkunits:
        for u in l:
            citing_pmids[u['uid']] = None
# print(citing_pmids.keys())

INFO: pmid 27569062
INFO: pmid 24283439
ERROR: problem retrieving citations for 24283439
INFO: pmid 22098123
INFO: pmid 21062266
INFO: pmid 20518782
ERROR: problem retrieving citations for 20518782
INFO: pmid 17184278
ERROR: problem retrieving citations for 17184278
INFO: pmid 15096145
ERROR: problem retrieving citations for 15096145
INFO: pmid 10333626
ERROR: problem retrieving citations for 10333626
INFO: pmid 7917994
ERROR: problem retrieving citations for 7917994
INFO: pmid 1619641
INFO: pmid 1842204
ERROR: problem retrieving citations for 1842204
INFO: pmid 22946697
INFO: pmid 3789005
ERROR: problem retrieving citations for 3789005
INFO: pmid 3712393
INFO: pmid 4697981
INFO: pmid 15490149
INFO: pmid 15490149
INFO: pmid 11078563
ERROR: problem retrieving citations for 11078563
INFO: pmid 22837569
INFO: pmid 19047498
ERROR: problem retrieving citations for 19047498
INFO: pmid 16825433
INFO: pmid 12522802
ERROR: problem retrieving citations for 12522802
INFO: pmid 12494447
ERROR: pro

In [5]:
all_pmids = seed_pmid_L + list(cited_in_pmids.keys()) + list(citing_pmids.keys())

In [6]:
len(all_pmids)

2958

#### The next cells connect to an instance of SemMedDB running on the same server as this notebook. Below is query for predications extracted from the articles in the set of PMIDS you picked, citing artcles, and articles cited by.  

**NOTE:** The cell that runs the actual semmeddb query will likely run for more than 2 hours. This is because there are over 90 million predications in the database.

**NOTE:** The VPN is likely to drop during that time. If that happens, you can reconnect to the VPN and then click on the red button that shows up in the upper right of the notebook indicating that the notebook is not connected. That will reset the connection. Since the notebook kernel is actually running on the server, all of the data in the notebook from previous cells should still be available. Sometimes, the mysql connection will 'go away' (die). You can re-run the block below to reconnect mysql if you need to.  

In [7]:
import MySQLdb  
cnx = MySQLdb.connect(user='semmeddb', passwd='semmeddb',
                              host='127.0.0.1',
                              db='semmeddb')
cursor = cnx.cursor()

In [8]:
## We store the PMIDs list in a table on the database 
import os
cursor.execute('''DROP TABLE IF EXISTS t_pmids_{};'''.format(os.getenv('JUPYTERHUB_USER')))

table_description = '''
  CREATE TABLE IF NOT EXISTS `t_pmids_{}`(
    `pmid` char(25) NOT NULL
  ) ENGINE=InnoDB DEFAULT CHARACTER SET latin1 COLLATE=latin1_swedish_ci;'''.format(os.getenv('JUPYTERHUB_USER'))

try:
    print("Creating temp table for pmid list")
    cursor.execute(table_description)
except:
    print("ERROR: Table creation failed")
else:
    print("OK")

Creating temp table for pmid list
OK


In [9]:
for pmid in all_pmids:    
    add_pmid = ('''INSERT INTO t_pmids_{} (pmid) VALUES ({})'''.format(os.getenv('JUPYTERHUB_USER'),pmid))
    cursor.execute(add_pmid)
    
# Make sure data is committed to the database
cnx.commit()

In [10]:
## This is the main query - it joins the SemMedDB predication table with the PMIDs of interest. 
## NOTE: this query can take more than 2 hours to run
smdb_q = ''' 
select p.predication_id,
       p.sentence_id,
       null as p_index, 
       p.PMID as pmid, 
       null as subject_source, 
       SUBSTRING_INDEX(p.SUBJECT_CUI,'|',1) as subject_cui, 
       p.SUBJECT_NAME as subject_name,
       p.SUBJECT_SEMTYPE as subject_type,
       null as subject_score,
       p.PREDICATE as predicate,
       null as object_source,
       SUBSTRING_INDEX(p.OBJECT_CUI,'|',1) as object_cui,
       p.OBJECT_NAME as object_name,
       p.OBJECT_SEMTYPE as object_type,
       null as object_score,
       0.8 as belief,
       null as sentence,
       'semmedb' as source
from semmeddb.PREDICATION p
where p.PMID in (select pmid from t_pmids_{})
;
'''.format(os.getenv('JUPYTERHUB_USER'))
cursor.execute(smdb_q)
data = cursor.fetchall()

In [11]:
## Confirm that the results were returned and save to a pickle file
import pickle
if len(data) > 0:
    print('INFO: SemMedDB Query returned {} predications.'.format(len(data)))
    f = open('semmedb-results.pickle','wb')
    pickle.dump(data,f)
    f.close()
    print('INFO: SemMedDB results saved to semmedb-results.pickle')
else:
    print('INFO: No SemMedDB results returned. If the previous query block ran without error, check if the PMIDs collection and loading blocks worked')

INFO: SemMedDB Query returned 10757 predications.
INFO: SemMedDB results saved to semmedb-results.pickle


In [12]:
## If the query as successful, then we can drop the teporary table
cursor.execute('''DROP TABLE t_pmids_{};'''.format(os.getenv('JUPYTERHUB_USER')))

0

#### The next cells give some summary information about the predications extracted from the cells above. Take a look at some of the predications and answer the questions that follow. 

In [13]:
import random
sampleIdxL = random.sample(range(0,len(data)), 100)

In [14]:
## Obtain predication score data 
scoreData = []
for idx in sampleIdxL:
    cursor.execute('''SELECT * FROM PREDICATION_AUX WHERE PREDICATION_ID = {};'''.format(data[idx][0]))
    scoreRec = cursor.fetchall()
    scoreData.append(scoreRec[0])

In [15]:
## Obtain the sentences from which predications were read by machine reader
sentenceData = []
for idx in sampleIdxL:
    cursor.execute('''SELECT * FROM SENTENCE WHERE SENTENCE_ID = {};'''.format(data[idx][1]))
    sentenceRec = cursor.fetchall()
    sentenceData.append(sentenceRec[0])

In [16]:
print(scoreData[0])

(55833977, 55833976, 'brain', 1, 1, 949, 954, 1000, 'PREP', 942, 944, 'Mitogen       Activated Protein Kinase', 1, 3, 887, 925, 876, datetime.datetime(2018, 3, 6, 12, 25, 6))


In [17]:
colLabs = ('predication_id','sentence_id','index','pmid','subject_source','subject_cui','subject_name','subject_type','subject_score','predicate','object_source','object_cui','object_name','object_type','object_score','belief','sentence','source')
scoreLabs = ('predication_aux_id','predication_id','subject_text','subject_dist','subject_maxdist','subject_start_index','subject_end_index','subject_score','indicator_type','predicate_start_index','predicate_end_index','object_text','object_dist','object_maxdist','object_start_index','object_end_index','object_score','curr_timestamp')
sentenceLabs = ('sentence_id','pmid','type','number','section_header','sentence','normalized_section_header')

ctr = 0
for i in sampleIdxL:
    r = zip(colLabs,data[i])
    print('\n\nPREDICTION ' + str(i))
    print('\n\t'.join([str(x) for x in tuple(r)]))
   
    t = zip(sentenceLabs,sentenceData[ctr])
    print('\nSENTENCE DATA:')
    print('\n\t'.join([str(x) for x in tuple(t)]))
    
    s = zip(scoreLabs,scoreData[ctr])
    print('\nSCORE DATA:')
    print('\n\t'.join([str(x) for x in tuple(s)]))
    
    
    print('--------------------------------------------------------------------------------\n\n')
    ctr += 1



PREDICTION 10583
('predication_id', 55833976)
	('sentence_id', 96726013)
	('index', None)
	('pmid', '16271875')
	('subject_source', None)
	('subject_cui', 'C0006104')
	('subject_name', 'Brain')
	('subject_type', 'bpoc')
	('subject_score', None)
	('predicate', 'LOCATION_OF')
	('object_source', None)
	('object_cui', 'C0752312')
	('object_name', 'Mitogen-Activated Protein Kinases')
	('object_type', 'aapp')
	('object_score', None)
	('belief', Decimal('0.8'))
	('sentence', None)
	('source', 'semmedb')

SENTENCE DATA:
('sentence_id', 96726013)
	('pmid', '16271875')
	('type', 'ab')
	('number', 4)
	('section_header', '')
	('sentence', 'Here, we identify lovastatin as a potent inhibitor of p21Ras/Mitogen       Activated Protein Kinase (MAPK) activity in the brain.')
	('normalized_section_header', '')

SCORE DATA:
('predication_aux_id', 55833977)
	('predication_id', 55833976)
	('subject_text', 'brain')
	('subject_dist', 1)
	('subject_maxdist', 1)
	('subject_start_index', 949)
	('subject_end_in

In [18]:
cursor.close()
cnx.close()

In [19]:
[predication_id,sentence_id,index,pmid,subject_source,subject_cui,subject_name,subject_type,subject_score,predicate,object_source,object_cui,object_name,object_type,object_score,belief,sentence,source] = range(0,18)
summaryD = {
    'subject_entities':0,
    'subject_types':0,
    'predicate_types':0,
    'object_entities':0,
    'object_types':0,
    }
for t in data:
    if not summaryD.get('s' + t[subject_cui]):
        summaryD['s' + t[subject_cui]] = True
        summaryD['subject_entities'] += 1
    if not summaryD.get('s' + t[subject_type]):
        summaryD['s' + t[subject_type]] = True
        summaryD['subject_types'] += 1
    if not summaryD.get(t[predicate]):
        summaryD[t[predicate]] = True
        summaryD['predicate_types'] += 1
    if not summaryD.get('o' + t[object_cui]):
        summaryD['o' + t[object_cui]] = True
        summaryD['object_entities'] += 1
    if not summaryD.get('o' + t[object_type]):
        summaryD['o' + t[object_type]] = True
        summaryD['object_types'] += 1
print('subject_entities: {}\nsubject_types: {}\npredicate_types: {}\nobject_entities: {}\nobject_types: {}'.format(summaryD['subject_entities'],summaryD['subject_types'],summaryD['predicate_types'],summaryD['object_entities'],summaryD['object_types']))       

subject_entities: 2137
subject_types: 95
predicate_types: 45
object_entities: 1738
object_types: 91


##### E4 - Answer the following questions (NOTE: you can read more about SemMedDB and how it was built here - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3509487/): 

1. Did you find any predications that make sense to you? Are they what you would expect for the domain of discourse you chose? List a few of them and explain why you think they make sense.

ANSWER: 

2. In your own words, what is a potential use for the semantic types assigned to the subject and object entities?

ANSWER: 

3. How are the predicate types different from the semantic types assigned to subject and object entities?

ANSWER: 

4. Did you find any predications that did not make sense? List them and explain why they do not make sense.

ANSWER: 

5. What is the NLP tool used to extract these predications? In a sentence or two, describe the approach that was used to train the NLP tool. Can you think of any limitations that the approach might place on the predication set you just queried?

ANSWER: 

6. Each predication has scores for the subject and object entities (see the score data) and a belief score (see the predication data). The belief score indicates a prior probability of the predication being correct given the sentence. What do the subject and object entity scores represent?
 
ANSWER: 

### The next cells using a forward chaining reasoner to apply simple  closure to the predications extracted above

In [20]:
## Set up the CLIPS environment
import clips
env = clips.Environment()

MIN_PREDICATION_BELIEF = 0.8
MIN_TRANSITIVE_BELIEF = 0.6  # chosen because it retains most depth 1 transitive inferences over semmed  

## NOTE: This clears test-inference.ntriples each time this cell is ran
## This accomplishes that
f = open("test-inference.ntriples",'w')
f.close()

#### We set the  rules up first. Look at the code and comments and then answer the questions below.

In [21]:
## First, we clear the working memory of the rule engine. This removes all Working Memory Elements and resets the 
## inference algorithm
env.clear()
env.reset()

## We will be writing the triples to RDF. This file is opened by the rule enginein append mode 
env.eval('(open "test-inference.ntriples" writeFile "a")')


## We build a template/frame for object-attribute-value entities. There is a slot to track if the oav object is 
## based on inference. There is also a slot to track a belief score which accumulates uncertainty. the predNS
## slot is to track the namespace of the oav entity which helps with translating the results to RDF
env.build("""
(deftemplate oav
 (slot object)
 (slot attribute)
 (slot value)
 (slot predNS)
 (slot inferred (default No))
 (slot belief (default 0.0)))
""")

## Here is the first rule - transitive closure. Notice that it only applies to a subset of 
## predicate types - predisposes, causes, influence, precedes, and part_of
## These correspond to predicate properties defined in the Relation Ontology
## The 'test' ensures that the total belief score for the entity is above the the threshold
## set above
env.build("""
(defrule transitive
  "a simple transitivity rule"
  (oav (object ?o)
       (attribute ?pred&:(member$ ?pred (create$ predisposes causes influence precedes part_of)))
       (value ?s)
       (predNS RO)
       (inferred No)
       (belief ?b1))
  (oav (object ?s)
       (attribute ?pred)
       (value ?q)
       (predNS RO)
       (inferred No)
       (belief ?b2))
   (test (>= (* ?b1 ?b2) {}))
  =>
  (assert (oav (object ?o)
               (attribute ?pred)
               (value ?q)
               (inferred Yes)
               (predNS RO)
               (belief (* ?b1 ?b2))))
  
  (printout writeFile (format nil "<http://dikb.org/cfc#%s> <http://purl.obolibrary.org/obo/%s> <http://dikb.org/cfc#%s>.%n" ?o ?pred ?q))   
)
""".format(MIN_TRANSITIVE_BELIEF))
# NOTE: add this line to RHS to see the belief scores:
# (printout writeFile (format nil "b1: %f, b2: %f, belief: %f>.%n" ?b1 ?b2 (* ?b1 ?b2))) 



## Here is the second rule - symmetric closure. Like the first rule, it only operates on certain
## predicate types
env.build("""
(defrule symmetric
  "a simple symmetry rule"
  (oav (object ?o)
       (attribute ?pred&:(member$ ?pred (create$ interacts_with association associated_with coexists_with compared_with same_as)))
       (value ?s)
       (predNS RO)
       (inferred No)
       (belief ?b))
  =>
  (assert (oav (object ?s)
               (attribute ?pred)
               (value ?o)
               (inferred Yes)
               (predNS RO)
               (belief ?b)))
  
  (printout writeFile (format nil "<http://dikb.org/cfc#%s> <http://purl.obolibrary.org/obo/%s> <http://dikb.org/cfc#%s>.%n" ?o ?pred ?s))  
)
""")


**E5** - Questions about the closure rules:

1. What happens to working memory when the RHS of the rule (the 'assert' statement) triggers?

ANSWER: 

2. The reasoner will run the rules over data in working memory (loaded by the cells below) using 'forward chaining'. Describe what that actually means. 

ANSWER:


3. What has to be true for reasoning come to a halt?

ANSWER:


4. Symmetric and transitive closure are very simple forms of logical inference. Can you think of additional rules that would help to ensure that the resulting knowledge base contains additional entailed facts of interest? 

ANSWER: 

5. COMMENT: One of the advantages of using a rule based programming approach to conduct graph closure is that the rules can be very modular and little or no thought needs to be put into how the rules interact with each other. It is difficult to see that in this toy example but imagine that there exists hundreds of rules to apply various other inferences in addition to these closures (e.g. by applying valid argument forms and use case specific reasoning). In that case, the rules could be maintained separately from the code as declarative knowledge objects. The principle that the rules should be modular becomes a great advantage for growing and maintainnig the system. 



### The next cells parse the SemMedDb predications and load them as facts  into working memory

In [22]:
originalTriplesF = open('original-triples.ntriples','w')

resourceD = {}
resourceDinv = {}
rcnt = 0
fctStrD = {}
semTypeD = {}
labelsD = {}

[PREDICATION_ID,SENTENCE_ID,INDEX,PMID,SUBJECT_SOURCE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_TYPE,SUBJECT_SCORE,PREDICATE,OBJECT_SOURCE,OBJECT_CUI,OBJECT_NAME,OBJECT_TYPE,OBJECT_SCORE,BELIEF,SENTENCE,SOURCE] = range(0,18)

## Uses the serialized predications from the cells above
f = open('semmedb-results.pickle','rb')
stmtTpls = pickle.load(f)
f.close()

for stmtSplt in stmtTpls: 
         
    if len(stmtSplt) != 18:
        continue
        
    if float(stmtSplt[BELIEF]) < MIN_PREDICATION_BELIEF:
        continue
    
    (subj, pred, obj) = (stmtSplt[SUBJECT_CUI],
                        stmtSplt[PREDICATE].lower().strip(),
                        stmtSplt[OBJECT_CUI])
        
    # Subj and obj need to be validly formatted CUIs
    if subj == None or obj == None or subj == "" or obj == "" or subj[0] != 'C' or obj[0] != 'C':
        continue

    # only write out and/or do inference over some predicates
    if pred not in ['affects','associated_with','augments',
                    'causes','coexists_with','complicates',
                    'disrupts','inhibits','interacts_with',
                    'part_of','precedes', 'predisposes','prevents',
                    'treats',
                    'association', 
                    'influence', 
                    'modification',
                    'regulateactivity',
                    'regulateamount',
                    'acetylation',
                    'deacetylation',
                    'defarnesylation',
                    'degeranylgeranylation',
                    'deglycosylation',
                    'dehydroxylation',
                    'demethylation'
                    'demyristoylation',
                    'depalmitoylation',
                    'dephosphorylation',
                    'deribosylation',
                    'desumoylation',
                    'deubiquitination',
                    'farnesylation',
                    'geranylgeranylation',
                    'glycosylation',
                    'hydroxylation',
                    'methylation',
                    'myristoylation',
                    'palmitoylation',
                    'phosphorylation',
                    'ribosylation',
                    'sumoylation',
                    'ubiquitination',
                    'activation','inhibition','increaseamount','decreaseamount'                    
                   ]:        
        continue
        
    # write the original triple to file, regardless of the predicate
    originalTriplesF.write("<http://dikb.org/cfc#{}> <http://purl.obolibrary.org/obo/{}> <http://dikb.org/cfc#{}>.\n".format(subj,pred,obj))
        
    # Track the subject and objet names
    subjName = stmtSplt[SUBJECT_NAME]
    objName = stmtSplt[OBJECT_NAME]
    if not labelsD.get(subjName):
        labelsD[subj] = subjName
    
    if not labelsD.get(objName):
        labelsD[obj] = objName
    
    # Track the semantic types
    semTypesStr = stmtSplt[SUBJECT_TYPE]
    if semTypesStr.find('[') == -1:
        if not semTypeD.get(subj):
            semTypeD[subj] = [semTypesStr.strip()]
        else:
            semTypeD[subj] = semTypeD[subj].append(semTypesStr.strip())
    else:
        semTypesStr = semTypesStr.replace("'",'').replace('[','').replace(']','')
        semTypesL = [x.strip() for x in semTypesStr.split(',')]
        if not semTypeD.get(subj):
            semTypeD[subj] = semTypesL
        else:
            semTypeD[subj] = semTypeD[subj] + semTypesL
            
    semTypesStr = stmtSplt[OBJECT_TYPE]
    if semTypesStr.find('[') == -1:
        if not semTypeD.get(obj):
            semTypeD[obj] = [semTypesStr.strip()]
        else:
            semTypeD[obj] = semTypeD[obj].append(semTypesStr.strip())
    else:
        semTypesStr = semTypesStr.replace("'",'').replace('[','').replace(']','')
        semTypesL = [x.strip() for x in semTypesStr.split(',')]
        if not semTypeD.get(obj):
            semTypeD[obj] = semTypesL
        else:
            semTypeD[obj] = semTypeD[obj] + semTypesL
        
    if not resourceD.get(subj):
        resourceD[subj] = 'r{}'.format(rcnt)
        resourceDinv['r{}'.format(rcnt)] = subj
        rcnt += 1
    
    if not resourceD.get(obj):
        resourceD[obj] = 'r{}'.format(rcnt)
        resourceDinv['r{}'.format(rcnt)] = obj
        rcnt += 1    
    
        
    fctStr = """
(oav (object {})
     (attribute {})
     (value {})
     (predNS {})
     (belief {})
)""".format(resourceD[subj], pred, resourceD[obj], 'RO', float(stmtSplt[BELIEF]))
    
    if not fctStrD.get(fctStr): 
        env.assert_string(fctStr)
        fctStrD[fctStr] = 1

# write the human readable labels as triples
for (e,st) in labelsD.items():
    if st:
        originalTriplesF.write('<http://dikb.org/cfc#{}> <http://www.w3.org/2000/01/rdf-schema#label> "{}".\n'.format(e,st.replace('"','')))

# Write the semantic types as triples
for (e,st) in semTypeD.items():
    if st:
        stSet = set(st)
        for elt in stSet:
            originalTriplesF.write("<http://dikb.org/cfc#{}> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://umls.org/st/#{}>.\n".format(e,elt.replace('"','')))
    
originalTriplesF.close()

#### This shows how many predications were loaded into working memory from the original set of predications after filtering out predicate types that we don't want in the knowledge graph

In [23]:
len(fctStrD.keys())

3563

#### The next cell shows how the data loaded into working memory is formatted

In [24]:
i = 0
for fact in env.facts():
    print(fact)
    if i == 20:
        break
    i += 1

(initial-fact)
(oav (object r0) (attribute part_of) (value r1) (predNS RO) (inferred No) (belief 0.8))
(oav (object r2) (attribute associated_with) (value r3) (predNS RO) (inferred No) (belief 0.8))
(oav (object r0) (attribute causes) (value r3) (predNS RO) (inferred No) (belief 0.8))
(oav (object r2) (attribute associated_with) (value r4) (predNS RO) (inferred No) (belief 0.8))
(oav (object r2) (attribute associated_with) (value r5) (predNS RO) (inferred No) (belief 0.8))
(oav (object r6) (attribute coexists_with) (value r7) (predNS RO) (inferred No) (belief 0.8))
(oav (object r4) (attribute coexists_with) (value r7) (predNS RO) (inferred No) (belief 0.8))
(oav (object r8) (attribute causes) (value r9) (predNS RO) (inferred No) (belief 0.8))
(oav (object r10) (attribute causes) (value r7) (predNS RO) (inferred No) (belief 0.8))
f-10    (oav (object r11) (attribute affects) (value r12) (predNS RO) (inferred No) (belief 0.8))
f-11    (oav (object r11) (attribute associated_with) (value 

#### The next cell runs the inference algorithm over the predications loaded into working memory. The count that is printed is the number of inferrences from the reasoner

In [25]:
env.run()  

2073

In [26]:
env.eval("close(writeFile)")

'close'

### The next cells wrap normalize the identifiers and format original and inferred predications and while translating the newly extracted knowledge statements to RDF

The reason for this  step is that we plan to integrate the knowledge extracted from the literature in SemMedDB with a much larger ontology-based knowledge graph. The semantics of that larger system are based on OBO-Foundry ontologies and the data format is RDF. This is not always a requirement. There are other systems for representing knowledge  in graph form including Neo4J and Grakn which we do not have time to over in class.

In [27]:
# predicate to RO mapping 
predMapD = {
    'affects':'RO_0002211',
    'associated_with':'RO_0002610',
    'augments':'augments',
    'causes':'RO_0002501',
    'coexists_with':'RO_0002490',
    'complicates':'complicates',
    'disrupts':'disrupts',
    'inhibits':'RO_0002212',
    'interacts_with':'RO_0002434',
    'part_of':'BFO_0000050',
    'precedes':'BFO_0000063',
    'predisposes':'RO_0003302',
    'prevents':'RO_0002599',
    'stimulates':'RO_0002213',
    'treats':'RO_0002606',
    'influence':'RO_0002501',
    'association':'RO_0002610',
    'modification':'RO_0002436',
    'regulateactivity':'RO_0002436',
    'regulateamount':'RO_0002436',
    'phosphorylation':'RO_0002447',
    'dephosphorylation':'RO_0002436',
    'ubiquitination':'RO_0002436',
    'deubiquitination':'RO_0002436',
    'sumoylation':'RO_0002436',
    'desumoylation':'RO_0002436',
    'hydroxylation':'RO_0002436',
    'dehydroxylation':'RO_0002436',
    'acetylation':'RO_0002436',
    'deacetylation':'RO_0002436',
    'glycosylation':'RO_0002436',
    'deglycosylation':'RO_0002436',
    'farnesylation':'RO_0002436',
    'defarnesylation':'RO_0002436',
    'geranylgeranylation':'RO_0002436',
    'degeranylgeranylation':'RO_0002436',
    'palmitoylation':'RO_0002436',
    'depalmitoylation':'RO_0002436',
    'myristoylation':'RO_0002436',
    'demyristoylation':'RO_0002436',
    'ribosylation':'RO_0002436',
    'deribosylation':'RO_0002436',
    'methylation':'RO_0002436',
    'demethylation':'RO_0002436',
    'activation':'RO_0002429',
    'inhibition':'RO_0002212',
    'increaseamount':'RO_0002429',
    'decreaseamount':'RO_0002212'
}

import re

f = open('test-inference.ntriples','r')
buf = f.read()
f.close()
rsL = buf.split('\n')

#rgx.findall(s)
#[('http://purl.org/vocab/relationship/', 'influence')]

rgx = re.compile('(http://purl.obolibrary.org/obo/)([a-z_]+)')
for i in range(0,len(rsL)):
    if rsL[i] == "":
        continue
        
    ml = rgx.findall(rsL[i])
    if len(ml) != 1:
        print('ERROR: could not match on predicate regex: {}'.format(rsL[i]))
        continue
        
    (uri,predicate) = ml[0]
    rsL[i] = rsL[i].replace(predicate, predMapD[predicate])
    

f = open('inferred-transitive-and-symmetric.ntriples','w')
rgx = re.compile('#(r[0-9]+)')
for it in rsL:
    keyL = rgx.findall(it)
    newTr = it
    for k in keyL:
        if resourceDinv.get(k):
            newTr = newTr.replace('http://dikb.org/cfc#' + k, 'http://dikb.org/cfc#' + resourceDinv[k])            
        else:
            print('ERROR: key not found in resourceDinv: {}'.format(k))
    f.write(newTr + '\n')
f.close()


In [28]:
# Do the same RO mapping for the original predications
f = open('original-triples.ntriples','r')
buf = f.read()
f.close()
rsL = buf.split('\n')

rgx = re.compile('(http://purl.obolibrary.org/obo/)([a-z_]+)')
f = open('original-triples-RO-mapped.ntriples','w')
for i in range(0,len(rsL)):
    if rsL[i] == "":
        continue
        
    if rsL[i].find('<http://www.w3.org/2000/01/rdf-schema#label>') != -1 or rsL[i].find('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>') != -1:
        f.write(rsL[i] + '\n')    
        continue
        
    ml = rgx.findall(rsL[i])
    if len(ml) != 1:
        print('ERROR: could not match on predicate regex: {}'.format(rsL[i]))
        continue
        
    (uri,predicate) = ml[0]
    f.write(rsL[i].replace(predicate, predMapD[predicate]) + '\n')    
f.close()

### Running SPARQL over the original and inferred triples

In [2]:
## We load the original extracted predicates (filtered as above) into an 
## in-memory triple store. Notice that we load RO as well.

import rdflib

In [4]:
## The SPARQL query just pulls up to 100 predications and 
## the human readable labels. The output 
g = rdflib.Graph()
g.parse("original-triples-RO-mapped.ntriples", format='nt')
print('Triple count after loading original-triples-RO-mapped.ntriples: {}'.format(len(g)))
g.parse("ro_with_imports_AD_mods.owl", format='xml')

qres = g.query("""
SELECT ?s_lab ?p_lab ?o_lab ?s_type ?o_type ?s ?p ?o
WHERE {{
  ?s ?p ?o.
  ?s rdfs:label ?s_lab.
  ?s rdf:type ?s_type.
  OPTIONAL{?p rdfs:label ?p_lab.}
  ?o rdfs:label ?o_lab.
  ?o rdf:type ?o_type.
 }}
LIMIT 100
""")

for row in qres:
    if row[5].find('cfc') == -1:
        continue
    print("\n\nTRIPLE:\n{}".format('\n'.join([str(x) for x in row])))

Triple count after loading original-triples-RO-mapped.ntriples: 6484


TRIPLE:
Cryptorchidism
existence overlaps
Neck webbing
http://umls.org/st/#cgab
http://umls.org/st/#cgab
http://dikb.org/cfc#C0010417
http://purl.obolibrary.org/obo/RO_0002490
http://dikb.org/cfc#C0221217


TRIPLE:
Cryptorchidism
existence overlaps
Bleeding tendency
http://umls.org/st/#cgab
http://umls.org/st/#dsyn
http://dikb.org/cfc#C0010417
http://purl.obolibrary.org/obo/RO_0002490
http://dikb.org/cfc#C1458140


TRIPLE:
Cryptorchidism
existence overlaps
Chest deformity
http://umls.org/st/#cgab
http://umls.org/st/#fndg
http://dikb.org/cfc#C0010417
http://purl.obolibrary.org/obo/RO_0002490
http://dikb.org/cfc#C0238983


TRIPLE:
Hyperextension
correlated with
Costello syndrome
http://umls.org/st/#sosy
http://umls.org/st/#dsyn
http://dikb.org/cfc#C0231451
http://purl.obolibrary.org/obo/RO_0002610
http://dikb.org/cfc#C0587248


TRIPLE:
Tissue specimen
part of
Malignant melanoma of conjunctiva
http://umls.org/st/#tisu


In [5]:
## We load the original AND inferred predications and run the same query. The print statement filters 
## out triples we have seen before from the orignal predications.
g = rdflib.Graph()
g.parse("original-triples-RO-mapped.ntriples", format='nt')
g.parse("inferred-transitive-and-symmetric.ntriples", format='nt')
print('Triple count after loading original and inferred-transitive-and-symmetric.ntriples: {}'.format(len(g)))
g.parse("ro_with_imports_AD_mods.owl", format='xml')

qres2 = g.query("""
SELECT ?s_lab ?p_lab ?o_lab ?s_type ?o_type ?s ?p ?o
WHERE {{
  ?s ?p ?o.
  ?s rdfs:label ?s_lab.
  ?s rdf:type ?s_type.
  OPTIONAL{?p rdfs:label ?p_lab.}
  ?o rdfs:label ?o_lab.
  ?o rdf:type ?o_type.
 }}
LIMIT 200
""")

for row in qres2:
    if row[5].find('cfc') == -1 or row in qres:
        continue
    print("\n\nTRIPLE:\n{}".format('\n'.join([str(x) for x in row])))

Triple count after loading original and inferred-transitive-and-symmetric.ntriples: 7234


TRIPLE:
Acid Phosphatase
negatively regulates
CRK protein, human|CRK
http://umls.org/st/#aapp
http://umls.org/st/#aapp
http://dikb.org/cfc#C0001109
http://purl.obolibrary.org/obo/RO_0002212
http://dikb.org/cfc#C1451465


TRIPLE:
Assessment procedure
is substance that treats
Participant
http://umls.org/st/#hlca
http://umls.org/st/#humn
http://dikb.org/cfc#C1261322
http://purl.obolibrary.org/obo/RO_0002606
http://dikb.org/cfc#C0679646


TRIPLE:
Assessment procedure
is substance that treats
Costello syndrome
http://umls.org/st/#hlca
http://umls.org/st/#dsyn
http://dikb.org/cfc#C1261322
http://purl.obolibrary.org/obo/RO_0002606
http://dikb.org/cfc#C0587248


TRIPLE:
Breast
part of
Human
http://umls.org/st/#bpoc
http://umls.org/st/#humn
http://dikb.org/cfc#C0006141
http://purl.obolibrary.org/obo/BFO_0000050
http://dikb.org/cfc#C0020114


TRIPLE:
Combined Modality Therapy
is substance that treats
Chron

**E6** - Use the next cells to write a SPARQL queries that count the number of distinct CUIs used for subject and object roles in the merged graph. Also, shown how the counts for subject and objects break down by semantic type. The basic structure of the queries are set up for you.

In [13]:
# Count distinct subject CUIs
qres = g.query("""
SELECT  ?s_type (count(distinct ?s) AS ?count)
WHERE {{
  ?s ?p ?o.
  ?s rdf:type ?s_type.
 }}
 group by ?s_type
""")
for row in qres:
    print("\n\nRESULT:\n{}".format('\n'.join([str(x) for x in row])))




RESULT:
http://www.w3.org/2002/07/owl#SymmetricProperty
77


RESULT:
http://www.w3.org/2002/07/owl#Class
118


RESULT:
http://www.w3.org/2002/07/owl#TransitiveProperty
47


RESULT:
http://www.w3.org/2002/07/owl#ObjectProperty
589


RESULT:
http://www.w3.org/2002/07/owl#Axiom
92


RESULT:
http://www.w3.org/2002/07/owl#AnnotationProperty
102


RESULT:
http://www.w3.org/2002/07/owl#Restriction
45


RESULT:
http://www.w3.org/2002/07/owl#NamedIndividual
5


RESULT:
http://www.w3.org/2002/07/owl#DatatypeProperty
1


RESULT:
http://www.w3.org/2002/07/owl#AsymmetricProperty
5


RESULT:
http://www.w3.org/2002/07/owl#IrreflexiveProperty
7


RESULT:
http://www.w3.org/2002/07/owl#FunctionalProperty
2


RESULT:
http://www.w3.org/2002/07/owl#Ontology
1


In [ ]:
# Count subject CUI grouped by semantic type
qres = g.query("""
SELECT  (count(distinct ?s) AS ?count)
WHERE {{
  ?s ?p ?o.
 }}
""")
for row in qres:
    print("\n\nRESULT:\n{}".format('\n'.join([str(x) for x in row])))


In [ ]:
# Count object CUIs
qres = g.query("""
SELECT <FILL IN>
WHERE {{
  <FILL IN>
 }}
""")
for row in qres:
    print("\n\nRESULT:\n{}".format('\n'.join([str(x) for x in row])))
    

In [14]:
# Count object CUI grouped by semantic type
qres = g.query("""
SELECT  ?o_type (count(distinct ?o) AS ?count)
WHERE {{
  ?s ?p ?o.
  ?o rdf:type ?o_type.
 }}
 group by ?o_type
""")
for row in qres:
    print("\n\nRESULT:\n{}".format('\n'.join([str(x) for x in row])))



RESULT:
http://www.w3.org/2002/07/owl#SymmetricProperty
73


RESULT:
http://www.w3.org/2002/07/owl#Class
107


RESULT:
http://www.w3.org/2002/07/owl#TransitiveProperty
37


RESULT:
http://www.w3.org/2002/07/owl#ObjectProperty
384


RESULT:
http://www.w3.org/2003/11/swrl#AtomList
66


RESULT:
http://www.w3.org/2002/07/owl#Restriction
44


RESULT:
http://www.w3.org/2003/11/swrl#IndividualPropertyAtom
55


RESULT:
http://umls.org/st/#mamm
10


RESULT:
http://umls.org/st/#neop
95


RESULT:
http://umls.org/st/#sosy
7


RESULT:
http://umls.org/st/#celf
28


RESULT:
http://umls.org/st/#humn
25


RESULT:
http://www.w3.org/2003/11/swrl#ClassAtom
11


RESULT:
http://umls.org/st/#dsyn
92


RESULT:
http://umls.org/st/#gngm
34


RESULT:
http://umls.org/st/#patf
27


RESULT:
http://umls.org/st/#fndg
21


RESULT:
http://umls.org/st/#orch
7


RESULT:
http://www.w3.org/2002/07/owl#AnnotationProperty
30


RESULT:
http://umls.org/st/#aapp
58


RESULT:
http://www.w3.org/2003/11/swrl#Variable
20


RESULT

In [18]:
## count subject and objects by predication type (RO)
qres = g.query("""
SELECT  ?p ?p_label (count(distinct ?s) AS ?s_count) (count(distinct ?o) AS ?o_count)
WHERE {{
  ?s ?p ?o.
  ?p rdfs:label ?p_label.
 }}
 group by ?p_label ?p 
""")
for row in qres:
    print("\n\nRESULT:\n{}".format('\n'.join([str(x) for x in row])))



RESULT:
http://purl.obolibrary.org/obo/RO_0002561
is symmetric relational form of process class
4
4


RESULT:
http://purl.obolibrary.org/obo/IAO_0000424
expand expression to
22
22


RESULT:
http://purl.obolibrary.org/obo/RO_0002211
regulates
278
292


RESULT:
http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym
has_broad_synonym
3
3


RESULT:
http://purl.obolibrary.org/obo/RO_0002434
interacts with
96
95


RESULT:
http://purl.obolibrary.org/obo/RO_0002560
is asymmetric relational form of process class
1
1


RESULT:
http://purl.obolibrary.org/obo/RO_0002212
negatively regulates
49
47


RESULT:
http://purl.obolibrary.org/obo/RO_0002501
causal relation between processes
145
143


RESULT:
http://purl.obolibrary.org/obo/IAO_0000426
first order logic expression
6
6


RESULT:
http://purl.obolibrary.org/obo/IAO_0000111
editor preferred term
16
16


RESULT:
http://purl.obolibrary.org/obo/IAO_0000116
editor note
74
79


RESULT:
http://purl.obolibrary.org/obo/RO_0002575
is direct form o

In [18]:
## count subject and objects by predication type (RO)
qres = g.query("""
SELECT  ?p ?p_label (count(distinct ?s) AS ?s_count) (count(distinct ?o) AS ?o_count)
WHERE {{
  ?s ?p ?o.
  ?p rdfs:label ?p_label.
 }}
 group by ?p_label ?p 
""")
for row in qres:
    print("\n\nRESULT:\n{}".format('\n'.join([str(x) for x in row])))



RESULT:
http://purl.obolibrary.org/obo/RO_0002561
is symmetric relational form of process class
4
4


RESULT:
http://purl.obolibrary.org/obo/IAO_0000424
expand expression to
22
22


RESULT:
http://purl.obolibrary.org/obo/RO_0002211
regulates
278
292


RESULT:
http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym
has_broad_synonym
3
3


RESULT:
http://purl.obolibrary.org/obo/RO_0002434
interacts with
96
95


RESULT:
http://purl.obolibrary.org/obo/RO_0002560
is asymmetric relational form of process class
1
1


RESULT:
http://purl.obolibrary.org/obo/RO_0002212
negatively regulates
49
47


RESULT:
http://purl.obolibrary.org/obo/RO_0002501
causal relation between processes
145
143


RESULT:
http://purl.obolibrary.org/obo/IAO_0000426
first order logic expression
6
6


RESULT:
http://purl.obolibrary.org/obo/IAO_0000111
editor preferred term
16
16


RESULT:
http://purl.obolibrary.org/obo/IAO_0000116
editor note
74
79


RESULT:
http://purl.obolibrary.org/obo/RO_0002575
is direct form o

#### There are many interesting patterns that we can apply to the extracted knowledge. I will discuss examples in class. This first query is  simple example that looks for a simple causal chain involving clinically relevant entities  

In [32]:
qres = g.query("""
SELECT ?s_lab ?p_lab ?o_lab ?p_lab ?d_lab ?s ?o ?d 
WHERE {{
  ?s rdf:type ?s_semType.FILTER(?s_semType != <http://umls.org/st/#bacs> && ?s_semType != <http://umls.org/st/#bhvr> && ?s_semType != <http://umls.org/st/#dora> && ?s_semType != <http://umls.org/st/#food> && ?s_semType != <http://umls.org/st/#hops> && ?s_semType != <http://umls.org/st/#horm> && ?s_semType != <http://umls.org/st/#clna> && ?s_semType != <http://umls.org/st/#nnon> && ?s_semType != <http://umls.org/st/#bpoc> && ?s_semType != <http://umls.org/st/#mamm> && ?s_semType != <http://umls.org/st/#bdsu> && ?s_semType != <http://umls.org/st/#qlco> && ?s_semType != <http://umls.org/st/#sosy> && ?s_semType != <http://umls.org/st/#mnob> && ?s_semType != <http://umls.org/st/#popg> && ?s_semType != <http://umls.org/st/#podg> && ?s != <http://dikb.org/cfc#C0338656> && ?s != <http://dikb.org/cfc#C0001811> && ?s != <http://dikb.org/cfc#C0002395> && ?s != <http://dikb.org/cfc#C0494463> && ?s != <http://dikb.org/cfc#C0567416> && ?s != <http://dikb.org/cfc#C0022023> && ?s != <http://dikb.org/cfc#C0025552> && ?s != <http://dikb.org/cfc#C0002520> && ?s != <http://dikb.org/cfc#C0033684> && ?s != <http://dikb.org/cfc#C0012854>) 

  ?o rdf:type ?o_semType.FILTER(?o != ?s && ?o_semType != <http://umls.org/st/#bacs> && ?o_semType != <http://umls.org/st/#bhvr> && ?o_semType != <http://umls.org/st/#dora> && ?o_semType != <http://umls.org/st/#food> && ?o_semType != <http://umls.org/st/#hops> && ?o_semType != <http://umls.org/st/#horm> && ?o_semType != <http://umls.org/st/#clna> && ?o_semType != <http://umls.org/st/#nnon> && ?o_semType != <http://umls.org/st/#bpoc> && ?o_semType != <http://umls.org/st/#mamm> && ?o_semType != <http://umls.org/st/#bdsu> && ?o_semType != <http://umls.org/st/#qlco> && ?o_semType != <http://umls.org/st/#sosy> && ?o_semType != <http://umls.org/st/#mnob> && ?o_semType != <http://umls.org/st/#popg> && ?o_semType != <http://umls.org/st/#podg> && ?o != <http://dikb.org/cfc#C0338656> && ?o != <http://dikb.org/cfc#C0001811> && ?o != <http://dikb.org/cfc#C0012854>) 

  ?s <http://purl.obolibrary.org/obo/RO_0003302> ?o.
  
  ?o <http://purl.obolibrary.org/obo/RO_0003302> ?d.

  ?s rdfs:label ?s_lab.
  
  OPTIONAL{<http://purl.obolibrary.org/obo/RO_0003302> rdfs:label ?p_lab.}
  
  ?o rdfs:label ?o_lab.
  
  ?d rdfs:label ?d_lab.
 }}
LIMIT 100
""")

for row in qres:
    
    print("\n\nTRIPLE:\n{}".format('\n'.join([str(x) for x in row])))



TRIPLE:
STAT3 gene|STAT3
causes or contributes to condition
Pathogenesis
causes or contributes to condition
Tumor Initiation
http://dikb.org/cfc#C1367307
http://dikb.org/cfc#C0699748
http://dikb.org/cfc#C0598935


TRIPLE:
STAT3 gene|STAT3
causes or contributes to condition
Pathogenesis
causes or contributes to condition
Malignant Neoplasms
http://dikb.org/cfc#C1367307
http://dikb.org/cfc#C0699748
http://dikb.org/cfc#C0006826


TRIPLE:
BRAF gene|BRAF|SNRPE
causes or contributes to condition
Pathogenesis
causes or contributes to condition
Tumor Initiation
http://dikb.org/cfc#C0812241
http://dikb.org/cfc#C0699748
http://dikb.org/cfc#C0598935


TRIPLE:
BRAF gene|BRAF|SNRPE
causes or contributes to condition
Pathogenesis
causes or contributes to condition
Malignant Neoplasms
http://dikb.org/cfc#C0812241
http://dikb.org/cfc#C0699748
http://dikb.org/cfc#C0006826


TRIPLE:
coenzyme Q10
causes or contributes to condition
Pathogenesis
causes or contributes to condition
Tumor Initiation
http://

#### This next query looks for a pattern typical of statistical confounders where there is a common cause to a disorder and a disease where the disorder is also causally related to the disease 

In [33]:

qres = g.query("""
SELECT ?s_lab ?p_lab ?o_lab ?s_lab ?p_lab ?d_lab ?o_lab ?p_lab ?d_lab ?s ?o ?d 
WHERE {{
  ?s rdf:type ?s_semType.FILTER(?s_semType != <http://umls.org/st/#bacs> && ?s_semType != <http://umls.org/st/#bhvr> && ?s_semType != <http://umls.org/st/#dora> && ?s_semType != <http://umls.org/st/#food> && ?s_semType != <http://umls.org/st/#hops> && ?s_semType != <http://umls.org/st/#horm> && ?s_semType != <http://umls.org/st/#clna> && ?s_semType != <http://umls.org/st/#nnon> && ?s_semType != <http://umls.org/st/#bpoc> && ?s_semType != <http://umls.org/st/#mamm> && ?s_semType != <http://umls.org/st/#bdsu> && ?s_semType != <http://umls.org/st/#qlco> && ?s_semType != <http://umls.org/st/#sosy> && ?s_semType != <http://umls.org/st/#mnob> && ?s_semType != <http://umls.org/st/#popg> && ?s_semType != <http://umls.org/st/#podg> && ?s != <http://dikb.org/cfc#C0338656> && ?s != <http://dikb.org/cfc#C0001811> && ?s != <http://dikb.org/cfc#C0002395> && ?s != <http://dikb.org/cfc#C0494463> && ?s != <http://dikb.org/cfc#C0567416> && ?s != <http://dikb.org/cfc#C0022023> && ?s != <http://dikb.org/cfc#C0025552> && ?s != <http://dikb.org/cfc#C0002520> && ?s != <http://dikb.org/cfc#C0033684> && ?s != <http://dikb.org/cfc#C0012854>) 

  ?o rdf:type ?o_semType.FILTER(?o != ?s && ?o_semType != <http://umls.org/st/#bacs> && ?o_semType != <http://umls.org/st/#bhvr> && ?o_semType != <http://umls.org/st/#dora> && ?o_semType != <http://umls.org/st/#food> && ?o_semType != <http://umls.org/st/#hops> && ?o_semType != <http://umls.org/st/#horm> && ?o_semType != <http://umls.org/st/#clna> && ?o_semType != <http://umls.org/st/#nnon> && ?o_semType != <http://umls.org/st/#bpoc> && ?o_semType != <http://umls.org/st/#mamm> && ?o_semType != <http://umls.org/st/#bdsu> && ?o_semType != <http://umls.org/st/#qlco> && ?o_semType != <http://umls.org/st/#sosy> && ?o_semType != <http://umls.org/st/#mnob> && ?o_semType != <http://umls.org/st/#popg> && ?o_semType != <http://umls.org/st/#podg> && ?o != <http://dikb.org/cfc#C0338656> && ?o != <http://dikb.org/cfc#C0001811> && ?o != <http://dikb.org/cfc#C0012854>) 

  ?s <http://purl.obolibrary.org/obo/RO_0003302> ?o.
  
  ?s <http://purl.obolibrary.org/obo/RO_0003302> ?d.
  
  ?o <http://purl.obolibrary.org/obo/RO_0003302> ?d.

  ?s rdfs:label ?s_lab.
  
  OPTIONAL{<http://purl.obolibrary.org/obo/RO_0003302> rdfs:label ?p_lab.}
  
  ?o rdfs:label ?o_lab.
  
  ?d rdfs:label ?d_lab.
 }}
LIMIT 100
""")

for row in qres:
    
    print("\n\nTRIPLE:\n{}".format('\n'.join([str(x) for x in row])))



TRIPLE:
STAT3 gene|STAT3
causes or contributes to condition
Pathogenesis
STAT3 gene|STAT3
causes or contributes to condition
Malignant Neoplasms
Pathogenesis
causes or contributes to condition
Malignant Neoplasms
http://dikb.org/cfc#C1367307
http://dikb.org/cfc#C0699748
http://dikb.org/cfc#C0006826


TRIPLE:
BRAF gene|BRAF|SNRPE
causes or contributes to condition
Pathogenesis
BRAF gene|BRAF|SNRPE
causes or contributes to condition
Malignant Neoplasms
Pathogenesis
causes or contributes to condition
Malignant Neoplasms
http://dikb.org/cfc#C0812241
http://dikb.org/cfc#C0699748
http://dikb.org/cfc#C0006826


TRIPLE:
coenzyme Q10
causes or contributes to condition
Pathogenesis
coenzyme Q10
causes or contributes to condition
Malignant Neoplasms
Pathogenesis
causes or contributes to condition
Malignant Neoplasms
http://dikb.org/cfc#C0056077
http://dikb.org/cfc#C0699748
http://dikb.org/cfc#C0006826


TRIPLE:
PTPN11 gene|PTPN11
causes or contributes to condition
Pathogenesis
PTPN11 gene|PTPN1

#### The above 2 queries only use the 'causes or contributes to condition' predicate type (RO_0003302) but there are several other predicate types that could be used to create patterns (see the RO predicates in your graph from one of the queries above). 

**E7** - Use the next cell block to write a query that uses a different predicate type than RO_0003302. What did you find?

In [17]:
qres = g.query("""
SELECT * 
WHERE {{
  ?s rdf:type ?s_semType.FILTER(?s_semType != <http://umls.org/st/#bacs> && ?s_semType != <http://umls.org/st/#bhvr> && ?s_semType != <http://umls.org/st/#dora> && ?s_semType != <http://umls.org/st/#food> && ?s_semType != <http://umls.org/st/#hops> && ?s_semType != <http://umls.org/st/#horm> && ?s_semType != <http://umls.org/st/#clna> && ?s_semType != <http://umls.org/st/#nnon> && ?s_semType != <http://umls.org/st/#bpoc> && ?s_semType != <http://umls.org/st/#mamm> && ?s_semType != <http://umls.org/st/#bdsu> && ?s_semType != <http://umls.org/st/#qlco> && ?s_semType != <http://umls.org/st/#sosy> && ?s_semType != <http://umls.org/st/#mnob> && ?s_semType != <http://umls.org/st/#popg> && ?s_semType != <http://umls.org/st/#podg> && ?s != <http://dikb.org/cfc#C0338656> && ?s != <http://dikb.org/cfc#C0001811> && ?s != <http://dikb.org/cfc#C0002395> && ?s != <http://dikb.org/cfc#C0494463> && ?s != <http://dikb.org/cfc#C0567416> && ?s != <http://dikb.org/cfc#C0022023> && ?s != <http://dikb.org/cfc#C0025552> && ?s != <http://dikb.org/cfc#C0002520> && ?s != <http://dikb.org/cfc#C0033684> && ?s != <http://dikb.org/cfc#C0012854>) 

  ?o rdf:type ?o_semType.FILTER(?o != ?s && ?o_semType != <http://umls.org/st/#bacs> && ?o_semType != <http://umls.org/st/#bhvr> && ?o_semType != <http://umls.org/st/#dora> && ?o_semType != <http://umls.org/st/#food> && ?o_semType != <http://umls.org/st/#hops> && ?o_semType != <http://umls.org/st/#horm> && ?o_semType != <http://umls.org/st/#clna> && ?o_semType != <http://umls.org/st/#nnon> && ?o_semType != <http://umls.org/st/#bpoc> && ?o_semType != <http://umls.org/st/#mamm> && ?o_semType != <http://umls.org/st/#bdsu> && ?o_semType != <http://umls.org/st/#qlco> && ?o_semType != <http://umls.org/st/#sosy> && ?o_semType != <http://umls.org/st/#mnob> && ?o_semType != <http://umls.org/st/#popg> && ?o_semType != <http://umls.org/st/#podg> && ?o != <http://dikb.org/cfc#C0338656> && ?o != <http://dikb.org/cfc#C0001811> && ?o != <http://dikb.org/cfc#C0012854>) 

  ?s <http://purl.obolibrary.org/obo/RO_0002434> ?o.
  
  ?s <http://purl.obolibrary.org/obo/RO_0002434> ?d.FILTER(?s != ?d && ?o != ?d)
  
  ?s rdfs:label ?s_lab.
  
  OPTIONAL{<http://purl.obolibrary.org/obo/RO_0002434> rdfs:label ?p_lab.}
  
  ?o rdfs:label ?o_lab.
  
  ?d rdfs:label ?d_lab.
 }}
LIMIT 100
""")

for row in qres:
    
    print("\n\nTRIPLE:\n{}".format('\n'.join([str(x) for x in row])))



TRIPLE:
PTPN11 gene|PTPN11
Orthologous Gene
interacts with
Ubiquinone
http://dikb.org/cfc#C0041536
http://umls.org/st/#gngm
http://dikb.org/cfc#C1335280
http://umls.org/st/#gngm
http://dikb.org/cfc#C1335144


TRIPLE:
MAP Kinase Gene
Neurofibromatosis Type 1 Protein
interacts with
Mitogen-Activated Protein Kinases
http://dikb.org/cfc#C0752312
http://umls.org/st/#aapp
http://dikb.org/cfc#C1370600
http://umls.org/st/#gngm
http://dikb.org/cfc#C0083725


TRIPLE:
MAP Kinase Gene
Neurofibromatosis Type 1 Protein
interacts with
ras Proteins
http://dikb.org/cfc#C0242853
http://umls.org/st/#aapp
http://dikb.org/cfc#C1370600
http://umls.org/st/#gngm
http://dikb.org/cfc#C0083725


TRIPLE:
MAP Kinase Gene
Neurofibromatosis Type 1 Protein
interacts with
Proteins
http://dikb.org/cfc#C0033684
http://umls.org/st/#aapp
http://dikb.org/cfc#C1370600
http://umls.org/st/#gngm
http://dikb.org/cfc#C0083725


TRIPLE:
insulin receptor substrate 1 protein|IRS1
PTPN11 gene|PTPN11
interacts with
Mitogen-Activate

#### This final cell sets things up for the next part of the assignment (to be assigned next week). It creates mapping files that will be needed to work that crosses the extracted triples to OBO entity identifiers in PheKnowLator

In [ ]:
## create a file with triples to map the UMLS CUIs to the target ontologies used in the Pheknowlator KG ()
all_cui_to_codeD = {}
umls_mapped_cnt = 0
reach_mapped_cnt = 0
bioportal_mapped_cnt = 0

# start by loading all GO and HPO UMLS CUIs
# ex: C2247561|GO|GO:0034023|"5-(carboxyamino)imidazole ribonucleotide mutase activity"
f = open('cui_to_ontology_maps/go_hpo_maps.csv','r')
buf = f.read()
f.close
lns = buf.split('\n')
for ln in lns[1:]: # skip header
    if ln == "":
        continue
        
    (cui, ont, code, string) = ln.replace('""','').split('|')
    if not all_cui_to_codeD.get(cui):
        all_cui_to_codeD[cui] = ['http://purl.obolibrary.org/obo/' + code.replace(':','_')]
    else:
        all_cui_to_codeD[cui].append('http://purl.obolibrary.org/obo/' + code.replace(':','_'))

umls_mapped_cnt = len(all_cui_to_codeD.keys())
        
# add mappings to OBO from REACH 
# ex: quercetin	"('CHEBI', 'CHEBI:16243')"	"{'CHEBI': 'CHEBI:16243', 'TEXT': 'Quercetin', 'HMDB': 'HMDB0005794', 'MESH': 'D011794', 'PUBCHEM': '5280343', 'CAS': '117-39-5', 'DRUGBANK': 'DB04216', 'CHEMBL': 'CHEMBL50'}"	C0034392
f = open('cui_to_ontology_maps/reach_bio_mapping.tsv','r')
buf = f.read()
f.close
lns = buf.split('\n')
for ln in lns[1:]: # skip header
    if ln == "":
        continue
        
    (agent_name,grounding,other_mapping,cui) = ln.replace('"','').split('\t')
    
    (ont_str,code) = eval(grounding)
    if ont_str == None or (code != None and code.find(':') == -1):
        continue
    
    if not all_cui_to_codeD.get(cui):
        all_cui_to_codeD[cui] = ['http://purl.obolibrary.org/obo/' + code.replace(':','_')]
    else:
        all_cui_to_codeD[cui].append('http://purl.obolibrary.org/obo/' + code.replace(':','_'))

reach_mapped_cnt = len(all_cui_to_codeD.keys()) - umls_mapped_cnt
        
## now, add mappings that came from a query of the bioportal annotator using the 
## string labels of the UMLS CUIs from the MRCONSO table (see cui_to_ontology_maps/non_go_hpo_strings_annotator_query.curl)

# Bioportal annotator response
import json
f = open('cui_to_ontology_maps/non_go_hpo_strings_annotator_response.json','r')
map_d_l = json.load(f)
f.close()

# all none GO or HPO UMLS CUIs mentioned in the triples and the string labels
f = open('cui_to_ontology_maps/non_go_hpo_strings.csv','r')
buf = f.read()
f.close
lns = buf.split('\n')
strToCuiD = {}
cuiToStrD = {}
for ln in lns[1:]: # skip header
    if ln == "":
        continue
        
    (cui, s) = ln.replace('""','').split('|')
    strToCuiD[s.upper()] = cui
    cuiToStrD[cui] = s.upper()

for d in map_d_l:
    mapped_to = d['annotatedClass']['@id']
    for mapped_from_d in d['annotations']:
            if not strToCuiD.get(mapped_from_d['text'].upper()):
                continue
            else:
                cui = strToCuiD[mapped_from_d['text'].upper()]
                if not all_cui_to_codeD.get(cui):
                    all_cui_to_codeD[cui] = [mapped_to]
                else:
                    if mapped_to not in all_cui_to_codeD[cui]:
                        all_cui_to_codeD[cui].append(mapped_to)

bioportal_mapped_cnt = len(all_cui_to_codeD.keys()) - reach_mapped_cnt
                        
print('CUI mapping total: {} ({} from umls, {} from reach-to-obo, {} from bioportal)'.format(len(all_cui_to_codeD.values()),
                                                                                            umls_mapped_cnt,
                                                                                            reach_mapped_cnt,
                                                                                            bioportal_mapped_cnt))
ontCounts = {}
for cui,m in all_cui_to_codeD.items():
    for elt in m:
        ontId = elt.split('/')[-1].split('_')[0]
        if ontCounts.get(ontId):
            ontCounts[ontId] += 1
        else:
            ontCounts[ontId] = 1

for ontId,count in ontCounts.items():
        print('{}:{}'.format(ontId,count))
        

f = open('cui_mapping_triples.ntriples','w')
for cui,m in all_cui_to_codeD.items():
    for elt in m:
        f.write("<http://dikb.org/cfc#{}> <http://dikb.org/cfc#obo_mapping> <{}>.\n".format(cui,elt))
f.close()        

#### Notice how many different OBO ontologies the CUIs map to. There also is very likely some loss of information since CUI to OBO mappings are incomplete. We will discuss more about this in class.